## This notebook requires Photutils 0.3

In [ ]:
import photutils
photutils.__version__

<img src="data/photutils_banner.svg">

## Photutils

- Code: https://github.com/astropy/photutils
- Documentation: http://photutils.readthedocs.org/en/stable/
- Issue Tracker:  https://github.com/astropy/photutils/issues

## Photutils Overview

- Background and background noise estimation
- Source Detection and Extraction
  - DAOFIND and IRAF's starfind
  - **Image segmentation**
  - local peak finder
- **Aperture photometry**
- PSF photometry
- PSF matching
- Centroids
- Morphological properties



## Preliminaries

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['image.cmap'] = 'viridis'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

### Load the data

We'll start by reading data and error arrays from FITS files.  These are cutouts from the Extreme-Deep Field (XDF) taken with WFC3/IR in the F160W filter.

In [ ]:
from astropy.io import fits
sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'
sci_hdulist = fits.open(sci_fn)
rms_hdulist = fits.open(rms_fn)

sci_hdulist[0].header['BUNIT'] = 'electron/s'

Print some info about the data.

In [ ]:
sci_hdulist.info()

Define the data and error arrays.

In [ ]:
data = sci_hdulist[0].data.astype(np.float)
error = rms_hdulist[0].data.astype(np.float)

Extract the data header and create a WCS object.

In [ ]:
from astropy.wcs import WCS

hdr = sci_hdulist[0].header
wcs = WCS(hdr)

Display the data.

In [ ]:
from astropy.visualization import scale_image
plt.imshow(scale_image(data, scale='sqrt', percent=99.5))

## Part 1:  Aperture Photometry

Photutils provides the following aperture classes, defined in pixel coordinates:

* `CircularAperture`
* `CircularAnnulus`

* `EllipticalAperture`
* `EllipticalAnnulus`

* `RectangularAperture`
* `RectangularAnnulus`

Along with variants of each, defined in celestial coordinates:

* `SkyCircularAperture`
* `SkyCircularAnnulus`

* `SkyEllipticalAperture`
* `SkyEllipticalAnnulus`

* `SkyRectangularAperture`
* `SkyRectangularAnnulus`

## Methods for handling aperture/pixel intersection

The are three methods for handling the aperture overlap with the pixel grid.

<img src="data/photutils_aperture_methods.svg">

### Perform circular-aperture photometry on some sources in the XDF

First, we define a circular aperture at a given position and radius.

In [ ]:
from photutils import CircularAperture

position = (90.73, 59.43)
radius = 5.
aperture = CircularAperture(position, r=radius)

Now perform photometry on the data using the `aperture_photometry()` function.  
**The default aperture method is 'exact'.**

In [ ]:
from photutils import aperture_photometry

phot = aperture_photometry(data, aperture)
phot

The output is an Astropy `QTable` (Quantity Table).  The table also contains metadata.

In [ ]:
phot.meta

Aperture photometry using the **'center'** method:

In [ ]:
phot = aperture_photometry(data, aperture, method='center')
phot

Aperture photometry using the **'subpixel'** method with `subpixels=5`:

This is equivalent to SExtractor aperture photometry.

In [ ]:
phot = aperture_photometry(data, aperture, method='subpixel', subpixels=5)
phot

We can input an error array to get the photometric errors.

In [ ]:
phot = aperture_photometry(data, aperture, error=error)
phot

The error array in the FITS file represents only the background error.  If we want to include
the Poisson error of the source we need to calculate the **total** error:

$\sigma_{\mathrm{tot}} = \sqrt{\sigma_{\mathrm{b}}^2 +
                  \frac{I}{g}}$
                  
where $\sigma_{\mathrm{b}}$ is the background-only error,
$I$ are the data values, and $g$ is the "effective gain".

The "effective gain" is the value (or image) needed to convert the data image to count units (e.g. electrons or photons), where Poisson statistics apply.

In [ ]:
# this time include the Poisson error of the source
from photutils.utils import calc_total_error

eff_gain = hdr['TEXPTIME']
tot_error = calc_total_error(data, error, eff_gain)

phot = aperture_photometry(data, aperture, error=tot_error)
phot

We can also input the data (and error) units via the `unit` keyword.

In [ ]:
# input the data units
import astropy.units as u

unit = u.electron / u.s
phot = aperture_photometry(data, aperture, error=tot_error, unit=unit)
phot

`Quantity` inputs for data and error are also allowed.

In [ ]:
phot = aperture_photometry(data * unit, aperture, error=tot_error * u.adu)
phot

The `unit` will not override the data or error unit.

In [ ]:
phot = aperture_photometry(data * unit, aperture, error=tot_error * u.adu, unit=u.photon)
phot

## Performing aperture photometry at multiple positions

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.
apertures = CircularAperture(positions, r=radius)

phot = aperture_photometry(data, apertures, error=tot_error, unit=unit)
phot

## Bad pixel masking

In [ ]:
# create a bad pixel
data2 = data.copy()
y, x = 59, 91
data2[y, x] = 100.

aperture_photometry(data2, apertures, error=tot_error)

Now mask the bad pixel so that it does not contribute to the photometry.

In [ ]:
mask = np.zeros_like(data2, dtype=bool)
mask[y, x] = True

aperture_photometry(data2, apertures, error=tot_error, mask=mask)

## Adding columns to the photometry table

Calculate the signal-to-noise (SNR) ratio and add it to the table.

In [ ]:
snr = phot['aperture_sum'] / phot['aperture_sum_err']

phot['snr'] = snr
phot

Calculate the F160W AB magnitude and add it to the table.

In [ ]:
f160w_zpt = 25.9463

# NOTE that the log10() function can be applied only to dimensionless quantities
abmag = -2.5 * np.log10(phot['aperture_sum'].value) + f160w_zpt

phot['abmag'] = abmag
phot

Calculate the ICRS Right Ascension and Declination and add them to the table.

In [ ]:
from astropy.wcs.utils import pixel_to_skycoord

# convert pixel positions to sky coordinates
x, y = np.transpose(positions)
coord = pixel_to_skycoord(x, y, wcs)

phot['ra_icrs'] = coord.icrs.ra
phot['dec_icrs'] = coord.icrs.dec
phot

Example:  write the table to an ASCII file in ECSV format:

In [ ]:
phot.write('my_photometry.txt', format='ascii.ecsv')

In [ ]:
!cat my_photometry.txt

Now read the table in from the (ecsv) ASCII file:

In [ ]:
from astropy.table import QTable
tbl = QTable.read('my_photometry.txt', format='ascii.ecsv')
tbl

In [ ]:
tbl.meta

In [ ]:
tbl['aperture_sum']

## Performing aperture photometry at multiple positions using multiple apertures - see extended notebook

## Aperture photometry using Sky apertures

First, let's define the sky coordinates by converting our pixel coordinates.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
x, y = np.transpose(positions)
coord = pixel_to_skycoord(x, y, wcs)
coord

Now define a circular aperture in sky coordinates.

For sky apertures, the aperture radius must be a `Quantity`, in either pixel or angular units.

In [ ]:
from photutils import SkyCircularAperture

radius = 5. * u.pix
sky_apers = SkyCircularAperture(coord, r=radius)
sky_apers.r

In [ ]:
radius = 0.5 * u.arcsec
sky_apers = SkyCircularAperture(coord, r=radius)
sky_apers.r

When using a sky aperture in angular units, `aperture_photometry` needs the WCS transformation.

In [ ]:
# via the wcs keyword
phot = aperture_photometry(data, sky_apers, wcs=wcs)
phot

In [ ]:
# or via a FITS hdu (i.e. header and data)
phot = aperture_photometry(sci_hdulist[0], sky_apers)
phot

## Encircled flux - see extended notebook

Here we want to perform aperture photometry at a single position with *many* apertures.

Instead of generating a big table, we'll simply loop over the apertures and extract the fluxes from individual tables.

## Local background estimation

First, let's create circular and circular-annulus apertures at the same positions. The main difference here is comparing apertures with different areas.

In [ ]:
from photutils import CircularAnnulus

positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]

aper = CircularAperture(positions, r=3)
bkg_aper = CircularAnnulus(positions, r_in=10., r_out=15.)
apers = [aper, bkg_aper]

Now, perform the photometry.

In [ ]:
phot = aperture_photometry(data, apers)
phot.rename_column('aperture_sum_0', 'aperture_sum')
phot.rename_column('aperture_sum_1', 'annulus_sum')
phot

# Caveat:  Quantity columns cannot be renamed

Note that the fluxes cannot be simply subtracted because the aperture areas are different.

First, calculate the mean background level (per pixel) in the annuli.

In [ ]:
phot['annulus_mean'] = phot['annulus_sum'] / bkg_aper.area()
phot

Then mulitply it by the circular aperture area.

In [ ]:
phot['aperture_bkg'] = phot['annulus_mean'] * aper.area()
phot

Now subtract the background.

In [ ]:
 # subtract the background
flux_bkgsub = phot['aperture_sum'] - phot['aperture_bkg']

phot['aperture_sum_bkgsub'] = flux_bkgsub
phot

## Apertures can plot themselves

In [ ]:
plt.imshow(scale_image(data, scale='sqrt', percent=98.))

aper.plot(color='white', lw=2)
bkg_aper.plot(color='white', lw=2, hatch='//', alpha=0.5)

## Part 2:  Image Segmentation

Image segmentation is the process where sources are identified and labeled in an image.

The sources are detected by using a S/N threshold level and defining the minimum number of pixels required within a source.

First, let's define a threshold image at 2$\sigma$ (per pixel) above the background.

In [ ]:
bkg = 0.  # background level in this image
nsigma = 2.
threshold = bkg + (nsigma * error)  # this should be background-only error

Now let's detect "8-connected" sources of size 5 pixels where each pixel is 2$\sigma$ above the background.

-connected pixels touch along their edges. 8-connected pixels touch along their edges or corners. For reference, SExtractor uses 8-connected pixels.

The result is a segmentation image (`SegmentationImage` object).  The segmentation image is the isophotal footprint of each source above the threshold.

In [ ]:
from photutils import detect_sources

npixels = 5
segm = detect_sources(data, threshold, npixels)

print('Found {0} sources'.format(segm.nlabels))

Display the segmentation image.

In [ ]:
from photutils.utils import random_cmap

rand_cmap = random_cmap(random_state=12345)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))
ax1.imshow(scale_image(data, scale='sqrt', percent=99.5))
ax1.set_title('Data')
ax2.imshow(segm, cmap=rand_cmap)
ax2.set_title('Segmentation Image')

We should filter (smooth) the data prior to source detection.

Let's use a 5x5 Gaussian kernel with a FWHM of 2 pixels.

In [ ]:
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma

sigma = 2.0 * gaussian_fwhm_to_sigma    # FWHM = 2 pixels
kernel = Gaussian2DKernel(sigma, x_size=5, y_size=5)
kernel.normalize()

ssegm = detect_sources(data, threshold, npixels, filter_kernel=kernel)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))
ax1.imshow(segm, cmap=rand_cmap)
ax1.set_title('Original Segmentation')
ax2.imshow(ssegm, cmap=rand_cmap)
ax2.set_title('Smoothed Segmentation Image')

### Source deblending

Note above that some of our detected sources were blended.  We deblend them using the `deblend_sources()` function, which uses a combination of multi-thresholding and watershed segmentation.

In [ ]:
from photutils import deblend_sources

segm2 = deblend_sources(data, segm, npixels, filter_kernel=kernel,
                        contrast=0.001, nlevels=32)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))
ax1.imshow(scale_image(data, scale='sqrt', percent=99.5))
ax1.set_title('Data')
ax2.imshow(segm2, cmap=rand_cmap)
ax2.set_title('Segmentation Image')

print('Found {0} sources'.format(segm2.max))

## Measure the photometry and morphological properties of detected sources -- see extended notebook

Please see the complete list of available [source properties](http://photutils.readthedocs.org/en/latest/api/photutils.segmentation.SourceProperties.html#photutils.segmentation.SourceProperties).

## Define the approximate isophotal ellipses for each object -- see the extended notebook

Create elliptical apertures for each object using the measured morphological parameters.